In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
    
    You are a helpful AI talking to a human.

    {chat_history}
    
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)


chain.predict(question="My name is geon")



> Entering new LLMChain chain...
Prompt after formatting:

    
    You are a helpful AI talking to a human.

    
    
    Human:My name is geon
    You:


> Finished chain.


'Hello Geon! How can I assist you today?'

In [2]:
chain.predict(question="I live in NamYangJu")




> Entering new LLMChain chain...
Prompt after formatting:

    
    You are a helpful AI talking to a human.

    Human: My name is geon
AI: Hello Geon! How can I assist you today?
    
    Human:I live in NamYangJu
    You:


> Finished chain.


"That's great! Namyangju is a beautiful city in South Korea. How can I assist you today, Geon?"

In [3]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    
    You are a helpful AI talking to a human.

    Human: My name is geon
AI: Hello Geon! How can I assist you today?
Human: I live in NamYangJu
AI: That's great! Namyangju is a beautiful city in South Korea. How can I assist you today, Geon?
    
    Human:What is my name?
    You:


> Finished chain.


'Your name is Geon.'